In [8]:
import sys
import os
import time
import pandas as pd 
import numpy as np
from collections import Counter
import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns',10000)
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_qry
import plot_tools as pt
import data_processing as dp


In [9]:
file_path = '/Users/yantingting/Seafile/风控/模型/13 菲律宾/线上模型复盘/老客户/'

In [19]:
my_df = pd.read_csv(file_path + 'my_df.csv')
df5 = pd.read_csv(file_path + 'df_fund_flag.csv')
my_df = pd.merge(my_df,df5[['order_no','apply_date','approved_principal','if_fund']],left_on = 'order_key', right_on = 'order_no', how = 'left')
my_df['apply_date'] = my_df['apply_time'].apply(lambda x: pd.to_datetime(x[:10]))
my_df['放款笔数_temp'] = my_df['放款笔数'].apply(lambda x: '>6' if x>6 else x)
my_df['上笔最大逾期天数_temp'] = my_df['上笔最大逾期天数'].apply(lambda x: '>6' if x>6 else x)
my_df['放款金额_temp'] = my_df['放款金额'].apply(lambda x: '>=5000' if x>=5000 else x)

In [20]:
my_df.head()

,order_key,apply_time,product_name,order_type,申请笔数,放款笔数,失败笔数,第一次还款距今天数,最近一次还款距今天数,累计逾期天数,最大逾期天数,最小逾期天数,最大展期次数,最大结清天数(展期算新的订单),逾期订单数,逾期次数(展期算新的订单),展期订单数,总展期次数,单笔最小结清天数,单笔最大结清天数,放款金额,等级,上笔放款金额,上笔逾期天数,上笔最大逾期天数,上笔最小逾期天数,上笔订单结清天数,上笔订单展期次数,order_no_x,effective_date,flag1,flag_max,flag_sum,order_no_y,apply_date,approved_principal,if_fund,放款笔数_temp,上笔最大逾期天数_temp,放款金额_temp
0,P2g201906250630,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,1,0,184,181,4,4,0,2,10,1.0,1.0,1.0,2,10,10,2000.0,0,1000.0,8,4,0,2,2,NaN,NaN,NaN,NaN,NaN,P2g201906250630,2019-06-25,2000.0,0,1,4,2000
1,P2g201907110205,2019-07-11 10:25:13,Peso2Go,QualityApplicationOrderClassC,2,1,1,200,197,4,4,0,2,10,1.0,1.0,1.0,2,10,10,3000.0,0,1000.0,8,4,0,2,2,NaN,NaN,NaN,NaN,NaN,P2g201907110205,2019-07-11,3000.0,0,1,4,3000
2,P2g201905020429,2019-05-02 14:01:10,Peso2Go,QualityApplicationOrderClassC,2,1,1,91,91,1,1,1,0,14,1.0,1.0,NaN,0,14,14,3000.0,0,3000.0,5,1,1,14,0,NaN,NaN,NaN,NaN,NaN,P2g201905020429,2019-05-02,3000.0,0,1,1,3000
3,P2g201905020860,2019-05-02 22:31:32,Peso2Go,QualityApplicationOrderClassC,3,1,2,91,91,1,1,1,0,14,1.0,1.0,NaN,0,14,14,3000.0,0,3000.0,5,1,1,14,0,NaN,NaN,NaN,NaN,NaN,P2g201905020860,2019-05-02,3000.0,0,1,1,3000
4,P2g201906260612,2019-06-26 14:20:15,Peso2Go,QualityApplicationOrderClassC,4,1,3,146,146,1,1,1,0,14,1.0,1.0,NaN,0,14,14,3000.0,0,3000.0,5,1,1,14,0,NaN,NaN,NaN,NaN,NaN,P2g201906260612,2019-06-26,3000.0,0,1,1,3000


In [21]:
df_fenbu = my_df[my_df['apply_date'] >='2019-11-23']
df_fenbu_fund = df_fenbu[df_fenbu['if_fund'] == 1]
df_train_test = my_df[(my_df['effective_date'] >='2019-09-05') & (my_df['effective_date']<='2019-12-15')]

In [25]:
remain_cols = ['order_key', 'apply_time','effective_date', '放款笔数_temp', '上笔最大逾期天数_temp','放款金额_temp','flag1']
df_fenbu_fund[remain_cols].to_excel(file_path + 'df_fenbu_fund.xlsx')
df_train_test[remain_cols].to_excel(file_path + 'df_train_test.xlsx')
df_fenbu[remain_cols].to_excel(file_path + 'df_fenbu.xlsx')

In [22]:
df_train_test.shape

(5352, 40)

In [35]:
writer = pd.ExcelWriter(file_path + 'my_result1226.xlsx')

In [7]:
table1 = pd.pivot_table(df_fenbu, index = '放款笔数_temp', columns='上笔最大逾期天数_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table1
table2 = pd.pivot_table(df_train_test, index = '放款笔数_temp', columns=['flag1','上笔最大逾期天数_temp'],values='order_key',aggfunc='count',fill_value=0,margins=True)
table2
df_train_test1 = df_train_test[df_train_test['flag1'] == 1]
table3 = pd.pivot_table(df_train_test1, index = '放款笔数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table3
table4 = pd.pivot_table(df_fenbu_fund, index = '放款笔数_temp', columns='上笔最大逾期天数_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table4
table2.to_csv(file_path + 'temp.csv')
# table1.to_excel(writer, sheet_name = '放款笔数(申请)')
# table2.to_excel(writer, sheet_name = '放款笔数(放款)') 
# table4.to_excel(writer, sheet_name = '放款笔数(申请_放款)') 

上笔最大逾期天数_temp,0,1,2,3,4,5,6,>6,All
放款笔数_temp,,,,,,,,,
1,1296,243,86,40,14,20,10,19,1728
2,632,86,35,11,3,2,0,4,773
3,195,28,13,4,1,2,0,1,244
4,57,14,4,3,0,0,0,2,80
5,34,6,9,0,0,0,0,1,50
6,11,4,0,0,0,0,0,0,15
>6,28,7,1,0,0,0,0,0,36
All,2253,388,148,58,18,24,10,27,2926


flag1           0.0                                 1.0                        \
上笔最大逾期天数_temp     0    1    2   3   4   5   6  >6     0    1    2   3   4   5   
放款笔数_temp                                                                       
1              1423  192   66  33   5  10   4  14   534  136   82  36  12  10   
2               594  107   42  13   0   3   1   3   230   58   30  20   7   1   
3               401   81   16  12   2   3   1   2   123   44   22  13   4   1   
4               210   61   21   4   3   1   2   2    65   19   14   3   2   3   
5               116   22   13   4   2   0   0   0    28   10    6   6   0   0   
6                79   20    4   3   1   1   1   0    16    8    5   1   0   0   
>6              103   18   11   5   1   1   1   1    14   12    5   2   0   0   
All            2926  501  173  74  14  19  10  22  1010  287  164  81  25  15   

flag1                  All  
上笔最大逾期天数_temp  6  >6        
放款笔数_temp                   
1              3  10  2570  
2              4   4  1117  
3              1   6   732  
4              0   1   411  
5              0   1   208  
6              0   0   139  
>6             0   1   175  
All            8  23  5352

放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
放款笔数_temp,,,,,,,
1,12,56,755,0,0,0,823
2,2,23,181,148,0,0,354
3,2,19,129,37,27,0,214
4,1,6,54,34,11,1,107
5,0,5,28,12,5,1,51
6,0,0,18,9,3,0,30
>6,0,2,17,12,3,0,34
All,17,111,1182,252,49,2,1613


上笔最大逾期天数_temp,0,1,2,3,4,5,6,>6,All
放款笔数_temp,,,,,,,,,
1,1232,230,80,40,6,7,5,7,1607
2,622,82,31,10,0,0,0,2,747
3,186,26,12,4,1,2,0,1,232
4,55,13,3,3,0,0,0,0,74
5,34,6,8,0,0,0,0,0,48
6,10,4,0,0,0,0,0,0,14
>6,26,6,1,0,0,0,0,0,33
All,2165,367,135,57,7,9,5,10,2755


In [8]:
table1 = pd.pivot_table(df_fenbu, index = '放款笔数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table1
table2 = pd.pivot_table(df_train_test, index = '放款笔数_temp', columns=['flag1','放款金额_temp'],values='order_key',aggfunc='count',fill_value=0,margins=True)
table2
df_train_test1 = df_train_test[df_train_test['flag1'] == 1]
table3 = pd.pivot_table(df_train_test1, index = '放款笔数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table3
table4 = pd.pivot_table(df_fenbu_fund, index = '放款笔数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table4
table1.to_excel(writer, sheet_name = '放款笔数(申请)')
table2.to_excel(writer, sheet_name = '放款笔数(放款)') 
table4.to_excel(writer, sheet_name = '放款笔数(申请_放款)') 

放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
放款笔数_temp,,,,,,,
1,25,139,1564,0,0,0,1728
2,7,44,61,661,0,0,773
3,2,7,34,35,166,0,244
4,2,6,12,15,18,27,80
5,1,2,14,9,14,10,50
6,0,0,4,1,6,4,15
>6,1,0,2,14,9,10,36
All,38,198,1691,735,213,51,2926


flag1        0.0                                       1.0                \
放款金额_temp 2000.0 3000.0 3500.0 4000.0 4500.0 >=5000 2000.0 3000.0 3500.0   
放款笔数_temp                                                                  
1             37    133   1577      0      0      0     12     56    755   
2             29     46    392    296      0      0      2     23    181   
3             16     40    301    106     55      0      2     19    129   
4              7     23    155     88     31      0      1      6     54   
5              6     13     92     37      8      1      0      5     28   
6              3     10     56     27     12      1      0      0     18   
>6             9     13     63     38     15      3      0      2     17   
All          107    278   2636    592    121      5     17    111   1182   

flag1                            All  
放款金额_temp 4000.0 4500.0 >=5000        
放款笔数_temp                             
1              0      0      0  2570  
2            148      0      0  1117  
3             37     27      0   732  
4             34     11      1   411  
5             12      5      1   208  
6              9      3      0   139  
>6            12      3      0   175  
All          252     49      2  5352

放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
放款笔数_temp,,,,,,,
1,12,56,755,0,0,0,823
2,2,23,181,148,0,0,354
3,2,19,129,37,27,0,214
4,1,6,54,34,11,1,107
5,0,5,28,12,5,1,51
6,0,0,18,9,3,0,30
>6,0,2,17,12,3,0,34
All,17,111,1182,252,49,2,1613


放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
放款笔数_temp,,,,,,,
1,24,99,1484,0,0,0,1607
2,6,36,55,650,0,0,747
3,2,7,30,35,158,0,232
4,0,4,11,15,18,26,74
5,1,1,13,9,14,10,48
6,0,0,4,1,5,4,14
>6,0,0,2,13,8,10,33
All,33,147,1599,723,203,50,2755


NameError: name 'writer' is not defined

In [37]:
table1 = pd.pivot_table(df_fenbu, index = '上笔最大逾期天数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table1
table2 = pd.pivot_table(df_train_test, index = '上笔最大逾期天数_temp', columns=('flag1','放款金额_temp'),values='order_key',aggfunc='count',fill_value=0,margins=True)
table2
df_train_test1 = df_train_test[df_train_test['flag1'] == 1]
table3 = pd.pivot_table(df_train_test1, index = '上笔最大逾期天数_temp', columns='放款金额_temp',values='order_key',aggfunc='count',fill_value=0,margins=True)
table3
table1.to_excel(writer, sheet_name = '上笔最大逾期天数(申请)')
table2.to_excel(writer, sheet_name = '上笔最大逾期天数(放款)') 

放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
上笔最大逾期天数_temp,,,,,,,
0,30,104,1295,614,168,42,2253
1,3,17,252,81,31,4,388
2,2,2,99,29,12,4,148
3,1,0,43,11,2,1,58
4,1,16,1,0,0,0,18
5,0,23,1,0,0,0,24
6,0,10,0,0,0,0,10
>6,1,26,0,0,0,0,27
All,38,198,1691,735,213,51,2926


flag1            0.0                                       1.0                \
放款金额_temp     2000.0 3000.0 3500.0 4000.0 4500.0 >=5000 2000.0 3000.0 3500.0   
上笔最大逾期天数_temp                                                                  
0                 73    173   2072    513     90      5     10     24    745   
1                 22     29    359     66     25      0      2     15    234   
2                  8     11    139     11      4      0      3      4    136   
3                  3      3     64      2      2      0      0      1     65   
4                  0     13      1      0      0      0      2     22      1   
5                  0     19      0      0      0      0      0     14      1   
6                  0      9      1      0      0      0      0      8      0   
>6                 1     21      0      0      0      0      0     23      0   
All              107    278   2636    592    121      5     17    111   1182   

flag1                                All  
放款金额_temp     4000.0 4500.0 >=5000        
上笔最大逾期天数_temp                             
0                198     31      2  3936  
1                 29      7      0   788  
2                 12      9      0   337  
3                 13      2      0   155  
4                  0      0      0    39  
5                  0      0      0    34  
6                  0      0      0    18  
>6                 0      0      0    45  
All              252     49      2  5352

放款金额_temp,2000.0,3000.0,3500.0,4000.0,4500.0,>=5000,All
上笔最大逾期天数_temp,,,,,,,
0,10,24,745,198,31,2,1010
1,2,15,234,29,7,0,287
2,3,4,136,12,9,0,164
3,0,1,65,13,2,0,81
4,2,22,1,0,0,0,25
5,0,14,1,0,0,0,15
6,0,8,0,0,0,0,8
>6,0,23,0,0,0,0,23
All,17,111,1182,252,49,2,1613


In [38]:
writer.save()